In [1]:
%pip install medviz

                                              0.0/47.1 kB ? eta -:--:--
     ---------------------------------------- 47.1/47.1 kB 2.3 MB/s eta 0:00:00
                                              0.0/1.7 MB ? eta -:--:--
     --------                                 0.4/1.7 MB 7.4 MB/s eta 0:00:01
     ---------------------------------------  1.7/1.7 MB 18.2 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 18.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import medviz as viz

viz.gif()

AttributeError: module 'medviz' has no attribute 'gif'